# Meningioma Experiment Replication

Start by cloning this repository and with reference to the README at the root of the repository, build the docker image and setup the jupyter notebook server to connect to this notebook.

In [5]:
# Ensure we are in the right directory
import os

os.chdir('/opt/project')
print(os.getcwd())

/opt/project


### Data Preparation
<a id='data-preparation'></a>
To replicate the experiment, you'll need T1, T2, T1 contrast enhanced (T1CE), Fluid-attenuated inversion recovery (FLAIR), and apparent diffusion coefficient (ADC) imaging in a `nii.gz` format. Additionally, a 3D segmentation mask in `.nii.gz` format is required.

First, preprocess the T1, T2, T1CE, and FLAIR images using the [Cancer Imaging Phenomics Toolkit with the BraTS Pre-processing Pipeline](https://cbica.github.io/CaPTk/preprocessing_brats.html) with no skull stripping or deep learning segmentation enabled. Assume your preprocessed T1, T2, T1CE, FLAIR, and unregistered ADC appear as follows in your directory:

radiomica_ai/\
│\
└── data/\
&emsp;│\
&emsp;└── meningioma_data/\
&emsp;&emsp;│\
&emsp;&emsp;└── ID_00001/\
&emsp;&emsp;&emsp;├── t1.nii.gz\
&emsp;&emsp;&emsp;├── t2.nii.gz\
&emsp;&emsp;&emsp;├── t1ce.nii.gz\
&emsp;&emsp;&emsp;├── flair.nii.gz\
&emsp;&emsp;&emsp;├── unregistered_adc.nii.gz\
&emsp;&emsp;&emsp;└── mask.nii.gz

Then, register the ADC imaging to T1CE using rigid transformation as below:

In [ ]:
from src.preprocessing import register_patients

register_patients(data_dir='./data/meningioma_data',
                  static_stem='t1ce.nii.gz',
                  moving_stem='unregistered_adc.nii.gz', 
                  output_stem='registered_adc.nii.gz', 
                  transform_method='rigid',
                  id_regex="ID_*")

### Replicating the pipeline
**Handcrafted radiomics only model**

`docker compose run app python main.py experiment=meningioma optimizer.n_trials=1000`

From which the best feature selection method was mrmr and best oversampling method was SMOTETomek

`docker compose run app python main.py experiment=meningioma optimizer.n_trials=500 split.method=repeated_stratified_kfold_no_test +split.n_repeats=20 "preprocessing.feature_selection_methods=[mrmr]" "preprocessing.oversampling_methods=[SMOTETomek]"`

**Handcrafted + deep learning radiomics model**

First download the [pretrained FMCIB model weights](https://zenodo.org/records/10528450/files/model_weights.torch?download=1), store it in outputs/pretrained_models/model_weights.torch, otherwise specify the path in preprocessing.autoencoder.module__weights_path argument

`docker compose run app python main.py experiment=meningioma_autoencoder optimizer.n_trials=1000`

From which the best feature selection method was lasso and best oversampling method was SMOTEENN

`docker compose run app python main.py experiment=meningioma_autoencoder optimizer.n_trials=500 split.method=repeated_stratified_kfold_no_test +split.n_repeats=20 "preprocessing.feature_selection_methods=[lasso]" "preprocessing.oversampling_methods=[SMOTEENN]"`

The results of the run can be seen in the mlflow dashboard by launching the mlflow server. 

# Inference on new data
To infer on new data using the pretrained pipeline, you'll need the zip file of the mlflow run folder. Download and extract the folder, paste the folder in `outputs/models/`. Then retrieve the run ID from mlflow

In [ ]:
HRO_run_id = 'af832aa5f6a74696b3b0e4f9c5034ffd'

The test directory is expected to have the same directory structure, and have already undergone the same steps in [Data Preparation](#data-preparation)

In [ ]:
from src.inference import Inferrer

inferrer = Inferrer(HRO_run_id,
                    image_stems=('registered_adc', 't2', 'flair', 't1', 't1ce'),
                    mask_stem='mask',
                    extraction_config='./conf/radiomic_params/meningioma_mr.yaml')

inferrer.predict('data/test_meningioma_data')